# GPT-J Finetuning

Copyright (c) 2023 Graphcore Ltd.

[GPT-J](https://huggingface.co/EleutherAI/gpt-j-6B) is a causal decoder-only transformer model which can be used for text-generation.
Causal means that a causal mask is used in the decoder attention, so that each token has visibility on previous tokens only.

Language models are very powerful because a huge variety of tasks can be formulated as a text-to-text problem and thus adapted to fit the generative setup, where the model is asked to correctly predict future tokens. This idea has been widely explored in [T5 paper: Exploring the Limits of Transfer Learning with a Unified
Text-to-Text Transformer](https://arxiv.org/pdf/1910.10683.pdf).

In this example we apply this idea and finetune GPT-J as a Causal Language Model (CLM) for Text Entailment on [GLUE MNLI dataset](https://huggingface.co/datasets/glue#mnli).

You can easily adapt this example to do your custom finetuning on several downstream tasks, such as Question Answering, Named Entity Recognition, Sentiment Analysis, Text Classification in general: you just need to prepare data in the right way.

Our weights are also available as an HF checkpoint at [Graphcore/gptj-mnli]( https://huggingface.co/Graphcore/gptj-mnli).

## Paperspace Setup

In [ ]:
%%capture
%pip install -r requirements.txt

In [ ]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))
if number_of_ipus != 16:
    raise ValueError(f"This example need 16 IPUs to work. Detected {number_of_ipus}")
    
os.environ["POPART_CACHE_DIR"] = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "cache")
checkpoint_dir =  os.getenv("CHECKPOINT_DIR", "checkpoints")

## Initial setup
### Finetuning configuration
First of all, we need to load the default configuration, defined in `config/finetuning_mnli.yml`.
These are optimised configuration to run the model on IPUs.
We need to pick the one suitable for a POD16.

This configuration uses a sequence length of 1024 tokens. GPT-J layers are split across 16 IPUs, using [Tensor Model Parallelism](https://arxiv.org/pdf/1909.08053.pdf). No data parallelism is used (This extra optimization is available when using a POD64).

The `gptj_fine_tuning_setup` setup the specified configuration, configures logging and Weight and Biases, and loads the Hugging Face pretrained model [EleutherAI/gpt-j-6B](https://huggingface.co/EleutherAI/gpt-j-6B).

In [ ]:
from utils.setup import gptj_fine_tuning_setup
from config import CONFIG_DIR

> **W&B**: We support logging to Weights & Biases.
If you want to use it, you will first need to manually log in (see the quickstart guide [here](https://docs.wandb.ai/quickstart)).


In [ ]:
# Set this to True if you want to use W&B. Be sure to be logged in.
wandb_setup = False

In [ ]:
# Choose a configuration
config, args, pretrained = gptj_fine_tuning_setup(
    CONFIG_DIR / "finetuning_mnli.yml", "release", "gptj_6B_1024_pod16", wandb_setup
)

In [ ]:
print(config.dumps_yaml())

### Validation Configuration
A default configuration for inference-only is available in `config/inference.yml`.
There are two such configurations:
- one is `gpt-j`. This is the base configuration, and is guaranteed to fit into memory with 1024 sequence length.
- `gpt-j-mnli` is slightly optimised for the mnli dataset. Indeed, the sequence length can be reduced and more batches can fit into memory.

In this example we will start from the default one, and manually reduce the sequence length and increase the batch size later on.
You can do the same on your custom dataset to find the best configuration.

In [ ]:
from utils.setup import gptj_config_setup

> **W&B** We support logging to Weights & Biases.
If you want to use it, you will first need to manually log in (see the quickstart guide [here](https://docs.wandb.ai/quickstart)).

In [ ]:
wandb_setup_on_eval = False

In [ ]:
eval_config, args, _ = gptj_config_setup(
    CONFIG_DIR / "inference.yml", "release", "gpt-j", hf_model_setup=False, wandb_setup=wandb_setup
)

In [ ]:
print(eval_config.dumps_yaml())

## Dataset
MNLI dataset consists of pairs of sentences, a *premise* and a *hypothesis*.
The task is to predict the relation between the premise and the hypothesis, which can be:
- `entailment`: hypothesis follows from the premise,
- `contradiction`: hypothesis contradicts the premise,
- `neutral`: hypothesis and premise are unrelated.

Data splits for the mnli dataset are the following:

|train |validation_matched|validation_mismatched|
|-----:|-----------------:|--------------------:|
|392702|              9815|                 9832|


You can explore it [on hugginface](https://huggingface.co/datasets/glue/viewer/mnli/train).
![MNLI dataset](imgs/mnli_dataset.png)


### Traning Preprocessing
The columns we are interested in are `hypothesis`, `premise` and `label`.

The first step consists in forming input prompts with the format
```bash
mnli hypothesis: {hypothesis} premise: {premise} target: {label} <|endoftext|>
```
For example:
```
mnli hypothesis: Product and geography are what make cream skimming work.  premise: Conceptually cream skimming has two basic dimensions - product and geography. target: neutral<|endoftext|>
```

Then, prompt sentences are tokenized and packed together to form 1024 token sequences, following [HF packing algorithm](https://github.com/huggingface/transformers/blob/v4.20.1/examples/pytorch/language-modeling/run_clm.py). No padding is used.

Finally, the prompt is split into `input_ids` and `labels`. The input consists of the full sentence but for the last token (`prompt[:-1]`), and the label is the sentence shifted by one (`prompt[1:]`).
Given the training format, no extra care is needed to account for different sequences: the model does not need to know which sentence a token belongs to.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import data.hf_data_utils as hf_data_utils
import data.mnli_data as mnli_data

The next two cells are the ones you want to change in a custom finetuning.

We first load the MNLI dataset, and then create a custom preprocessing function to build prompts suitable for a
text-to-text setup.
In a custom finetuning, you will need to choose a format for your prompts and change the `form_training_prompts` function.

In [ ]:
# Load HF dataset
dataset = load_dataset("glue", "mnli", split="train")

In [ ]:
print(dataset[0])

In [ ]:
# Form prompts in the format mnli hypothesis: {hypothesis} premise: {premise} target: {class_label} <|endoftext|>
def form_training_prompts(example):
    hypothesis = example["hypothesis"]
    premise = example["premise"]
    class_label = ["entailment", "neutral", "contradiction"][example["label"]]

    example["text"] = f"mnli hypothesis: {hypothesis} premise: {premise} target: {class_label}<|endoftext|>"
    return example

form_training_prompts(dataset[0]["text"])

In [ ]:
dataset = dataset.map(
    form_training_prompts,
    remove_columns=["hypothesis", "premise", "label", "idx"],
    load_from_cache_file=False,
    desc="Generating text prompt",
)

In [ ]:
# shows first textual prompt
print(dataset[0])

After that, we tokenize the prompts. You won't need to change this step in a custom finetuning.

In [ ]:
# Create tokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer.add_special_tokens({"pad_token": "<|extratoken_1|>"})  # index 50257

In [ ]:
# Tokenize prompts
dataset = dataset.map(
    mnli_data.tokenizes_text(tokenizer),
    batched=True,
    batch_size=1000,
    num_proc=1,
    remove_columns=dataset.column_names,
    load_from_cache_file=False,
    desc="Tokenizing text",
)

In [ ]:
# shows first tokenized prompt
print(dataset[0])

Finally, we use the HF packing algorithm (`group_text`) to create packed sentences of the specified sequence length,
and separate inputs and labels.
Again, this is a step you are not going to change in a custom finetuning.

In [ ]:
# Pack tokenized prompts into sequences and split sequences in input_ids and labels
dataset = dataset.map(
    hf_data_utils.group_texts(config),
    batched=True,
    batch_size=1000,
    num_proc=1,
    load_from_cache_file=False,
    desc="Packing sequences",
)

In [ ]:
print(len(dataset))

In [ ]:
# Show a portion of first sentence. You can see that the label is the input shifted by one.
print("first 10 tokens of first sentence")
print("input_ids")
print(dataset["input_ids"][0][:10])
print("labels - shifted by one")
print(dataset["labels"][0][:10])

> **Note** If you want to adapt this code to another dataset, be sure to call the inputs and labels in the same way: `input_ids` and `labels`.

### Validation Preprocessing
For validation, we use the [MNLI validation_mismatched](https://huggingface.co/datasets/glue/viewer/mnli_mismatched/validation) split.

"Mismatched" means validation examples are not derived from the same sources as those in the training set and therefore not closely resembling any of the examples seen at training time.

Similarly to what we did for training, the first preprocessing step is creating prompts, this time without including the answer.

In [ ]:
def form_validation_prompts(example):
    hypothesis = example["hypothesis"]
    premise = example["premise"]
    class_label = ["entailment", "neutral", "contradiction"][example["label"]]

    example["text"] = f"mnli hypothesis: {hypothesis} premise: {premise} target:"
    return example

In [ ]:
eval_dataset = load_dataset("glue", "mnli", split="validation_mismatched")
eval_dataset = eval_dataset.map(form_validation_prompts,
                                remove_columns=["hypothesis", "premise", "idx"],
                                load_from_cache_file=False,
                                desc="Generating text prompt",
                               )

In [ ]:
print(eval_dataset[0])

Finally, input prompts are tokenized.

In [ ]:
def prepare_validation_features(dataset, tokenizer):
    tokenized_examples = []
    for example in dataset["text"]:
        tokenized_example = tokenizer.encode(example, return_tensors="pt").squeeze()
        tokenized_examples.append(tokenized_example)
    return {"input_ids": tokenized_examples, "label": dataset["label"]}

eval_dataset = eval_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=eval_dataset.column_names,
    load_from_cache_file=False,
    fn_kwargs={"tokenizer": tokenizer},
)

In [ ]:
print(eval_dataset[0])

> **Note** If you want to adapt this code to another dataset, be sure to call the inputs and labels in the same way: `input_ids` and `label`.

## Customise configuration and create a Trainer

Right at the beginning of the notebook we have loaded some default configurations for training and inference. We are now going to show how to customise some of the parameters for your needs.

### Customise training configuration
In the cells below we list the parameters you are most likely to play around when doing a custom finetuning.

These are the training steps, dropout probability and optimizer/learning rate parameters.

Moreover, it is important that you specify **checkpoints** parameters, namely a folder to save the finetuned weights and a periodicity for checkpointing. Be aware that saving checkpoints takes time, so you don't want to save them too often.
To disable intermediate checkpoints set `config.checkpoint.steps = 0`. The final checkpoint is always saved provided the save directory is given. Set it to `None` if you don't want to save weights, but it's unlikely you want to disable the last checkpoint.

If you are not resuming training, and you don't care about resuming the training later on you can reduce the time and memory required to save checkpoints by specifying `optim_state=False`. In this case, only the model weights will be saved, while the optimiser state will be discarded.

Checkpoints will be saved in the directory given by the environment variable `CHECKPOINT_DIR`, which we saved in `checkpoint_dir` at the beginning. You can have a look to the path printing it out in the cell below.

In [ ]:
print(checkpoint_dir)

In [ ]:
# Customise training arguments
config.model.dropout_prob = 0.0 
config.training.steps = 400

In [ ]:
# Customise optimiser and learning rate schedule
config.training.optimizer.learning_rate.maximum = 5e-06
config.training.optimizer.learning_rate.warmup_proportion = 0.005995
config.training.optimizer.learning_rate.beta1 = 0.9
config.training.optimizer.learning_rate.beta2 = 0.999
config.training.optimizer.learning_rate.weight_decay = 0.0
config.training.optimizer.learning_rate.gradient_clipping = 1.0

In [ ]:
# Customise checkpoints
config.checkpoint.save = checkpoint_dir # where the model is saved. None means don't save any checkpoint.
config.checkpoint.steps = 100  # how often you save the model. 0 means only the final checkpoint is saved.
config.checkpoint.to_keep = 4  # maximum number of checkpoints kept on disk
config.checkpoint.optim_state = False # Whether to include the optimiser state in checkpoints.

In [ ]:
# Resume training
config.checkpoint.load = None # you can specify a directory containing a previous checkpoint,
                              # os.path.join(checkpoint_dir, ...)

### Customise validation configuration
For validation there are less relevant parameters.

You can control the maximum number of tokens generated by the model using the  `output_length` parameter. If you know that the targets are only few tokens long, it is convenient to set it to a small number. Generation stops if the output token is `<|endoftext|>` or after `output_length` tokens are generated.

In our case, we set the output_len to 5 to accommodate all class labels and the `<|endoftext|>` token.

You can also reduce the model `sequence_len` to account for the maximum length of sentences encountered in the validation dataset, plus the `output_length` specified.
In our case of MNLI example, `sequence_len = 229` and this allows us to increase the batch size to `16`.

If you adapt this example to another dataset, you can compare your dataset `max_len` with ours and decide if you can safetly use the increased batch size. Otherwise, stick with the default one or create your own.

You can specify a checkpoint to be used for validation. If `None` is provided, the latest weights will be used.

In [ ]:
from functools import reduce

In [ ]:
# Specify maximum number of tokens that can be generated.
eval_config.inference.output_length = 5

In [ ]:
# Reducing sequence length
max_len = reduce(lambda l, e: max(l, len(e["input_ids"])), eval_dataset, 0)

In [ ]:
print(f"Maximum length in mnli-mismatched dataset: {max_len}")

In [ ]:
eval_config.model.sequence_length = max_len + eval_config.inference.output_length

In [ ]:
print(f"Setting sequence length to {eval_config.model.sequence_length}")

In [ ]:
# You can specify a directory containing a previous checkpoint.
# None means latest weights are used
config.checkpoint.load = None # os.path.join(checkpoint_dir, ...)

In [ ]:
print(config.dumps_yaml())

Finally, we need to define a validation metric. We will use `accuracy`.

In [ ]:
import evaluate

In [ ]:
accuracy_metric = evaluate.load("accuracy")

We also need to define a function to convert our generated labels to integer indices.

In [ ]:
def postprocess_mnli_predictions(generated_sentences):
    labels_to_ids = {"entailment": 0, "neutral": 1, "contradiction": 2, "unknown": -1}
    predictions = []
    for s in generated_sentences:
        answer = mnli_data.extract_class_label(s)
        predictions.append(labels_to_ids[answer])
    return predictions

### Create a Trainer
Once a config is specified, we are ready to create the training session with the help of the 
`MNLIFinetuningTrainer` class.
You need to provide the following arguments:

- *config*: the training configuration.
- *pretrained*: the Hugging Face pre-trained model, used to initialise the weights.
- *dataset*: the training dataset.

Moreover, you can specify:

- *eval_dataset*: the validation dataset.
- *eval_config*: the inference configuration, to be used in validation.
- *tokenizer*: the tokenizer, needed by validation.
- *metric*: the metric for validation. An Hugging Face metric from `evaluate` module, or a custom function. We use the `accuracy` metric.
- *process_answers_func*: a function to convert the generated answers to the format required by the metric and the labels. For example we need to convert textual categories `[entailment, contradiction,neutral]` to indices.

These extra arguments can also be provided later on when calling `trainer.evaluate(...)`.

The first time you run this notebook, it will take around 10 minutes to compile the training model and about 4 minutes to compile the inference model.

In [ ]:
from utils.trainer import MNLIFinetuningTrainer

In [ ]:
trainer = MNLIFinetuningTrainer(config,
                                pretrained,
                                dataset,
                                eval_dataset,
                                eval_config,
                                tokenizer,
                                clf_metrics,
                                postprocess_mnli_predictions)

## Run Finetuning
We can now run training for the number of steps you set in the config.
Checkpoints will be saved in the folder you specified in `config.save`, with the periodicity identified by `config.checkpoint.steps`.

Training takes around 8 minutes to start because the pretrained weights need to be loaded to the IPUs.
After that, each step takes around 22 seconds, processing 5.7 samples/s.

Each sample consists of 1024 tokens, so the performance is 

$$
\text{Throughput} = 1024*5.7 \approx 5800 \, \text{tokens/s}
$$

You can estimate how much it would take to do finetuning on your dataset by plugging this velocity

$$
\text{time} = \frac{\text{dataset_size(tokens)}}{\text{tokens/s}} * \text{epochs}
$$

This time does not include checkpoint time.

In [ ]:
trainer.train()

## Run validation
Finally, we validate our model on [mnli-mismatched](https://huggingface.co/datasets/glue/viewer/mnli_mismatched/test) split of MNLI dataset.

Generative inference is performed token-by-token using a greedy heuristic: the next token is chosen based on the highest logits.

We run token-by-token inference in batches to generate labels for multiple examples at once.
We then compute the accurancy comparing the model answers with the true labels.

The resulting model matches SOTA performance with 82.5% accuracy.

```
Total number of examples                 9832
Number with badly formed result          0
Number with incorrect result             1725
Number with correct result               8107 
[82.5%]
 ```

In [ ]:
trainer.evaluate()

## Save HF checkpoint
You can save the trained weights so that they can be uploaded to Hugging Face and used in Hugging Face torch model.
You can specify a checkpoint path if you want to convert a specific checkpoint, instead of the latest weights.

In [ ]:
hf_checkpoint_path = "hf_checkpoint"
ckpt_path = None # os.path.join(checkpoint_dir, ...)

In [ ]:
finetuned = trainer.save_hf_checkpoint(hf_checkpoint_path, ckpt_path)

The same model can be later used with standard HF pipeline on any hardware.

```python
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-j-6B')
hf_model = AutoModelForCausalLM.from_pretrained("Graphcore/gptj-mnli", pad_token_id=tokenizer.eos_token_id)
generator =  pipeline('text-generation', model=hf_model, tokenizer=tokenizer)

prompt = "mnli hypothesis: Your contributions were of no help with our students' education." \
         "premise: Your contribution helped make it possible for us to provide our students with a quality education. target:"

out = generator(prompt, return_full_text=False, max_new_tokens=5, top_k=1)
# [{'generated_text': ' contradiction'}]
```